# Can You Make Money Betting on Nate Silver's Data?

## A Brief Experiment in Value-Betting the Moneyline - Ben Carneiro



#### Hypothesis: It's possible to bet with an aggregate statistical edge on NFL MoneyLines using 538 Data

Given the nature of a sportsbook- the bets on each side of an outcome must cancel each other out, to limit the sportsbook's exposure to risk.

A balanced sportsbook should sometimes contain "Good Odds" that will pay more often than the necessary win frequency to break even. 

My gut says that without such inefficiencies, the book would be unable to balance. Their existence being certain, however, says nothing to my ability in finding such inefficiencies. 

I'm hoping that good odds can be identified with little more than readily-available, public data- in this instance, Nate Silver's 538

### Data Downloads
<a href = "https://projects.fivethirtyeight.com/nfl-api/nfl_elo.csv" target="_blank"> 538 Data</a>    
<a href = "https://www.sportsbookreviewsonline.com/scoresoddsarchives/nfl/nfl%20odds%202020-21.xlsx" target="_blank"> Odds Data</a>

In [1]:
##  Import libraries and data

import pandas as pd, numpy as np
nate_silver = pd.read_csv(r'C:\Users\benca\Downloads\nfl-elo\nfl-elo\nfl_elo.csv')
odds2020 = pd.read_excel(r'C:\Users\benca\Downloads\nfl odds 2020-21 (1).xlsx')

## Create Sports-Betting Functions

#### Implied Odds
imp(moneyline_odds) - Returns required win-frequency to break even

#### To Decimal Odds
to_dec(moneyline_odds) - Returns decimal odds from moneyline odds (+200 = 3.0)

#### Expected Value
ev(ML,model,betsize) - Expected Value (EV) is your average expected return per dollar of risk, over an infinite number of bets

GIVEN that: 'model' (probability where 0<X<1) is perfectly accurate

#### Actual Value
av(dec,win) - Actual Return per dollar of risk on a specific bet or group of bets

In [2]:
## Calculates implied odds of a given moneyline
## What I call implied odds- appears to not be what most gamblers call implied odds-
## This function returns the frequency with which you must win a bet in order to break even

def imp(mline):
    global implied
    if mline > 0:
        implied = 100/(mline + 100)
    if mline < 0:
        implied = abs(mline)/(100 + abs(mline))
    return implied 

## Calculates Decimal Odds from Moneyline
## imp(z) = 1/to_dec(z)

def to_dec(mline):
    if mline > 0:
        return (mline/100 + 1)
    if mline < 0:
        return (100/abs(mline) + 1)
    
## Calculates Expected Value from a given moneyline, "real" probability, and betsize

def ev(mline, model, betsize=1):
    global implied
    if mline > 0:
        implied = 100/(mline + 100)
    if mline < 0:
        implied = abs(mline)/(100 + abs(mline))
    return (model * betsize * ((1/implied) - 1)) + ((model-1) * betsize)

## Calculates actual return from Decimal Odds and Result of Game

def av(dec, result):
    global send
    if result == 'W':
        send = dec-1
    if result == 'L':
        send = -1
    else: send = 0
    return send

## Cleaning and combining data

In [3]:
## Drop non-2020 data from 538 dataset

model2020 = nate_silver[nate_silver['season'] == 2020].reset_index(drop=True)

## Assign a unique ID to each game of the 2020 Season

model2020['id'] = model2020.index + 1

## Conform the date in the "Odds" data to the format found in the 538 Data

odds=odds2020
odds['new_date'] = ""

for n in odds.index:
    if odds['Date'][n] < 500:
        odds['new_date'][n] = '2021-0' + str(odds['Date'][n])[0] + "-" + str(odds['Date'][n])[1:3]
    elif odds['Date'][n] < 1000:
        odds['new_date'][n] = '2020-0' + str(odds['Date'][n])[0] + "-" + str(odds['Date'][n])[1:3]
    else:
        odds['new_date'][n] = '2020-' + str(odds['Date'][n])[0:2] + "-" + str(odds['Date'][n])[2:4]

## Conform the Names in the "Odds" data to the format found in the 538 Data
        
teamNames = {'Houston':'HOU', 'KansasCity':'KC', 'Miami':'MIA', 'NewEngland':'NE', 'Cleveland':'CLE',
       'Baltimore':'BAL', 'NYJets':'NYJ', 'Buffalo':'BUF', 'LasVegas':'OAK', 'Carolina':'CAR',
       'Seattle':'SEA', 'Atlanta':'ATL', 'Philadelphia':'PHI', 'Washington':'WSH', 'Chicago':'CHI',
       'Detroit':'DET', 'Indianapolis':'IND', 'Jacksonville':'JAX', 'GreenBay':'GB', 'Minnesota':'MIN',
       'LAChargers':'LAC', 'Cincinnati':'CIN', 'Arizona':'ARI', 'SanFrancisco':'SF', 'TampaBay':'TB',
       'NewOrleans':'NO', 'Dallas':'DAL', 'LARams':'LAR', 'Pittsburgh':'PIT', 'NYGiants':'NYG',
       'Tennessee':'TEN', 'Denver':'DEN', 'LVRaiders':'OAK', 'KCChiefs':'KC', 'Kansas':'KC', 'Tampa':'TB',
       'Washingtom':'WSH'}

odds = odds2020.replace(to_replace = teamNames)

<ipython-input-3-64987fe040ce>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds['new_date'][n] = '2020-0' + str(odds['Date'][n])[0] + "-" + str(odds['Date'][n])[1:3]
<ipython-input-3-64987fe040ce>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds['new_date'][n] = '2020-' + str(odds['Date'][n])[0:2] + "-" + str(odds['Date'][n])[2:4]
<ipython-input-3-64987fe040ce>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds['new_dat

In [4]:
## Using the new matching dates and names
## Assign the unique game ID from the 538 data to the odds data

odds['id'] = 0
for n in model2020.index:
    temp = odds[odds['new_date'] == model2020['date'][n]]
    temp = temp[(temp['Team'] == model2020['team1'][n]) | (temp['Team'] == model2020['team2'][n])]
    for x in temp.index:
        odds['id'][x] = n + 1

<ipython-input-4-84d3598e23d3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds['id'][x] = n + 1


In [5]:
## Actually combining the 2 dataframes

temp = model2020[['date', 'team1','elo_prob1','qbelo_prob1','id']]
temp2 = model2020[['date', 'team2','elo_prob2','qbelo_prob2','id']]
model = temp.rename(columns={"team1": "team", "elo_prob1": "elo", "qbelo_prob1":"qb"}).append(temp2.rename(columns={"team2": "team", "elo_prob2": "elo", "qbelo_prob2":"qb"}))
model = model.sort_values(by = ['id','team'])
model = model.reset_index(drop=True)
odds = odds.sort_values(by = ['id','Team'])
odds = odds.reset_index(drop=True)
nfl2020 = pd.concat([model,odds], axis=1)

## deleting the duplicate ID column
nfl2020 = nfl2020.loc[:,~nfl2020.columns.duplicated()]

## Calculations on combined data

In [6]:
## Creating Decimal Odds from MoneyLine
nfl2020['dec'] = nfl2020['ML'].apply(to_dec, convert_dtype=True)

## Creating Expected Value from the QB ELO probabilities
nfl2020['ev'] = np.vectorize(ev)(nfl2020['ML'], nfl2020['qb'])

## Creating Expected Value from the Regular ELO probabilities
nfl2020['ev2'] = np.vectorize(ev)(nfl2020['ML'], nfl2020['elo'])

In [7]:
## Assigning "W","L", or "T" to a column called ['win']

nfl2020['win'] = ''
for z in nfl2020.index:
    if (nfl2020.index[z] % 2 == 0):
        if (nfl2020['Final'][z] > nfl2020['Final'][(z+1)]):
            nfl2020['win'][z] = 'W'
        elif (nfl2020['Final'][z] == nfl2020['Final'][(z+1)]):
            nfl2020['win'][z] = 'T'
        else: nfl2020['win'][z] = 'L'
    else: 
        if (nfl2020['Final'][z] > nfl2020['Final'][(z-1)]):
            nfl2020['win'][z] = 'W'
        elif (nfl2020['Final'][z] == nfl2020['Final'][(z-1)]):
            nfl2020['win'][z] = 'T'
        else: nfl2020['win'][z] = 'L'

<ipython-input-7-bbc1b5010b38>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else: nfl2020['win'][z] = 'L'
<ipython-input-7-bbc1b5010b38>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl2020['win'][z] = 'W'
<ipython-input-7-bbc1b5010b38>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl2020['win'][z] = 'W'
<ipython-input-7-bbc1b5010b38>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [8]:
## Calculating ACTUAL VALUE / ACTUAL RETURN
## From decimal odds and game results
av = pd.DataFrame(data=[],columns=['av'])

for z in nfl2020.index:
    if (nfl2020['win'][z] == 'W'):
        temp = pd.DataFrame(data=[[nfl2020['dec'][z] - 1.0]],columns=['av'])
        av = pd.concat([av,temp], ignore_index=True)
    elif (nfl2020['win'][z] == 'L'):
        temp = pd.DataFrame(data=[[-1]],columns=['av'])
        av = pd.concat([av,temp], ignore_index=True)
    else: 
        temp = pd.DataFrame(data=[[0]],columns=['av'])
        av = pd.concat([av,temp], ignore_index=True)
nfl2020 = pd.concat([nfl2020,av], axis=1)

## Testing different strategies on the 2020 NFL Season

In [23]:
models = pd.DataFrame()

bank_all = 100.0
bet_size_all = 1.0
bets_taken_all = 0
df_all = pd.DataFrame(data=[],columns=['date','bank_all','bets_all'])

bank_dog = 100.0
bet_size_dog = 1.0
bets_taken_dog = 0
df_dog = pd.DataFrame(data=[],columns=['bank_dog','bets_dog'])

bank_fav = 100.0
bet_size_fav = 1.0
bets_taken_fav = 0
df_fav = pd.DataFrame(data=[],columns=['bank_fav','bets_fav'])

bank_evplus = 100.0
bet_size_evplus = 1.0
bets_taken_evplus = 0
df_evplus = pd.DataFrame(data=[],columns=['bank_evplus','bets_evplus'])

bank_evminus = 100.0
bet_size_evminus = 1.0
bets_taken_evminus = 0
df_evminus = pd.DataFrame(data=[],columns=['bank_evminus','bets_evminus'])

bank_ev5plus = 100.0
bet_size_ev5plus = 1.0
bets_taken_ev5plus = 0
df_ev5plus = pd.DataFrame(data=[],columns=['bank_ev5plus','bets_ev5plus'])

bank_ev10plus = 100.0
bet_size_ev10plus = 1.0
bets_taken_ev10plus = 0
df_ev10plus = pd.DataFrame(data=[],columns=['bank_ev10plus','bets_ev10plus'])

bank_model1 = 100.0
bet_size_model1 = 1.0
bets_taken_model1 = 0
df_model1 = pd.DataFrame(data=[],columns=['bank_model1','bets_model1'])

bank_model2 = 100.0
bet_size_model2 = 1.0
bets_taken_model2 = 0
df_model2 = pd.DataFrame(data=[],columns=['bank_model2','bets_model2'])

for z in nfl2020.index:
    
    bet_size_all=bank_all/100
    bets_taken_all +=1
    bank_all += bet_size_all * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[nfl2020['new_date'][z], bank_all, bets_taken_all]], columns=['date','bank_all','bets_all'])
    df_all = pd.concat([df_all,temp],ignore_index=True)
    
    if nfl2020['dec'][z] > 2:
        bet_size_dog=bank_dog/100
        bets_taken_dog +=1
        bank_dog += bet_size_dog * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_dog, bets_taken_dog]], columns=['bank_dog','bets_dog'])
    df_dog = pd.concat([df_dog,temp],ignore_index=True)
    
    if nfl2020['dec'][z] < 2:
        bet_size_fav=bank_fav/100
        bets_taken_fav +=1
        bank_fav += bet_size_fav * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_fav, bets_taken_fav]], columns=['bank_fav','bets_fav'])
    df_fav = pd.concat([df_fav,temp],ignore_index=True) 
    
    if nfl2020['ev'][z] > 0:
        bet_size_evplus=bank_evplus/100
        bets_taken_evplus +=1
        bank_evplus += bet_size_evplus * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_evplus, bets_taken_evplus]], columns=['bank_evplus','bets_evplus'])
    df_evplus = pd.concat([df_evplus,temp],ignore_index=True)
    
    if nfl2020['ev'][z] < 0:
        bet_size_evminus=bank_evminus/100
        bets_taken_evminus +=1
        bank_evminus += bet_size_evminus * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_evminus, bets_taken_evminus]], columns=['bank_evminus','bets_evminus'])
    df_evminus = pd.concat([df_evminus,temp],ignore_index=True)
    
    if nfl2020['ev'][z] > 0.05:
        bet_size_ev5plus=bank_ev5plus/100
        bets_taken_ev5plus +=1
        bank_ev5plus += bet_size_ev5plus * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_ev5plus, bets_taken_ev5plus]], columns=['bank_ev5plus','bets_ev5plus'])
    df_ev5plus = pd.concat([df_ev5plus,temp],ignore_index=True)
    
    if nfl2020['ev'][z] > 0.1:
        bet_size_ev10plus=bank_ev10plus/100
        bets_taken_ev10plus +=1
        bank_ev10plus += bet_size_ev10plus * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_ev10plus, bets_taken_ev10plus]], columns=['bank_ev10plus','bets_ev10plus'])
    df_ev10plus = pd.concat([df_ev10plus,temp],ignore_index=True)
    
    if (nfl2020['ev'][z] > -0.1) & (nfl2020['ev'][z] < 0.5) & (nfl2020['qb'][z] > nfl2020['elo'][z]):
        bet_size_model1=bank_model1/100
        bets_taken_model1 +=1
        bank_model1 += bet_size_model1 * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_model1, bets_taken_model1]], columns=['bank_model1','bets_model1'])
    df_model1 = pd.concat([df_model1,temp],ignore_index=True)
    
    if (nfl2020['ev'][z] > 0.05) & (nfl2020['ev'][z] < 0.3) :
        bet_size_model2=bank_model2/100
        bets_taken_model2 +=1
        bank_model2 += bet_size_model2 * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_model2, bets_taken_model2]], columns=['bank_model2','bets_model2'])
    df_model2 = pd.concat([df_model2,temp],ignore_index=True)    
    
models=pd.concat([df_all,df_dog,df_fav,df_evplus,df_evminus,df_ev5plus,df_ev10plus,df_model1,df_model2],axis=1)

models_agg = pd.DataFrame()

bank_all = 100.0
bet_size_all = 1.0
bets_taken_all = 0
df_all = pd.DataFrame(data=[],columns=['date','bank_all','bets_all'])

bank_dog = 100.0
bet_size_dog = 1.0
bets_taken_dog = 0
df_dog = pd.DataFrame(data=[],columns=['bank_dog','bets_dog'])

bank_fav = 100.0
bet_size_fav = 1.0
bets_taken_fav = 0
df_fav = pd.DataFrame(data=[],columns=['bank_fav','bets_fav'])

bank_evplus = 100.0
bet_size_evplus = 1.0
bets_taken_evplus = 0
df_evplus = pd.DataFrame(data=[],columns=['bank_evplus','bets_evplus'])

bank_evminus = 100.0
bet_size_evminus = 1.0
bets_taken_evminus = 0
df_evminus = pd.DataFrame(data=[],columns=['bank_evminus','bets_evminus'])

bank_ev5plus = 100.0
bet_size_ev5plus = 1.0
bets_taken_ev5plus = 0
df_ev5plus = pd.DataFrame(data=[],columns=['bank_ev5plus','bets_ev5plus'])

bank_ev10plus = 100.0
bet_size_ev10plus = 1.0
bets_taken_ev10plus = 0
df_ev10plus = pd.DataFrame(data=[],columns=['bank_ev10plus','bets_ev10plus'])

bank_model1 = 100.0
bet_size_model1 = 1.0
bets_taken_model1 = 0
df_model1 = pd.DataFrame(data=[],columns=['bank_model1','bets_model1'])

bank_model2 = 100.0
bet_size_model2 = 1.0
bets_taken_model2 = 0
df_model2 = pd.DataFrame(data=[],columns=['bank_model2','bets_model2'])

for z in nfl2020.index:
    
    bet_size_all=bank_all/50
    bets_taken_all +=1
    bank_all += bet_size_all * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[nfl2020['new_date'][z], bank_all, bets_taken_all]], columns=['date','bank_all','bets_all'])
    df_all = pd.concat([df_all,temp],ignore_index=True)
    
    if nfl2020['dec'][z] > 2:
        bet_size_dog=bank_dog/50
        bets_taken_dog +=1
        bank_dog += bet_size_dog * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_dog, bets_taken_dog]], columns=['bank_dog','bets_dog'])
    df_dog = pd.concat([df_dog,temp],ignore_index=True)
    
    if nfl2020['dec'][z] < 2:
        bet_size_fav=bank_fav/50
        bets_taken_fav +=1
        bank_fav += bet_size_fav * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_fav, bets_taken_fav]], columns=['bank_fav','bets_fav'])
    df_fav = pd.concat([df_fav,temp],ignore_index=True) 
    
    if nfl2020['ev'][z] > 0:
        bet_size_evplus=bank_evplus/50
        bets_taken_evplus +=1
        bank_evplus += bet_size_evplus * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_evplus, bets_taken_evplus]], columns=['bank_evplus','bets_evplus'])
    df_evplus = pd.concat([df_evplus,temp],ignore_index=True)
    
    if nfl2020['ev'][z] < 0:
        bet_size_evminus=bank_evminus/50
        bets_taken_evminus +=1
        bank_evminus += bet_size_evminus * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_evminus, bets_taken_evminus]], columns=['bank_evminus','bets_evminus'])
    df_evminus = pd.concat([df_evminus,temp],ignore_index=True)
    
    if nfl2020['ev'][z] > 0.05:
        bet_size_ev5plus=bank_ev5plus/50
        bets_taken_ev5plus +=1
        bank_ev5plus += bet_size_ev5plus * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_ev5plus, bets_taken_ev5plus]], columns=['bank_ev5plus','bets_ev5plus'])
    df_ev5plus = pd.concat([df_ev5plus,temp],ignore_index=True)
    
    if nfl2020['ev'][z] > 0.1:
        bet_size_ev10plus=bank_ev10plus/50
        bets_taken_ev10plus +=1
        bank_ev10plus += bet_size_ev10plus * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_ev10plus, bets_taken_ev10plus]], columns=['bank_ev10plus','bets_ev10plus'])
    df_ev10plus = pd.concat([df_ev10plus,temp],ignore_index=True)
    
    if (nfl2020['ev'][z] > -0.1) & (nfl2020['ev'][z] < 0.5) & (nfl2020['qb'][z] > nfl2020['elo'][z]):
        bet_size_model1=bank_model1/50
        bets_taken_model1 +=1
        bank_model1 += bet_size_model1 * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_model1, bets_taken_model1]], columns=['bank_model1','bets_model1'])
    df_model1 = pd.concat([df_model1,temp],ignore_index=True)
    
    if (nfl2020['ev'][z] > 0.05) & (nfl2020['ev'][z] < 0.3) :
        bet_size_model2=bank_model2/50
        bets_taken_model2 +=1
        bank_model2 += bet_size_model2 * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_model2, bets_taken_model2]], columns=['bank_model2','bets_model2'])
    df_model2 = pd.concat([df_model2,temp],ignore_index=True)    
    
models_agg=pd.concat([df_all,df_dog,df_fav,df_evplus,df_evminus,df_ev5plus,df_ev10plus,df_model1,df_model2],axis=1)

models_super_agg = pd.DataFrame()

bank_all = 100.0
bet_size_all = 1.0
bets_taken_all = 0
df_all = pd.DataFrame(data=[],columns=['date','bank_all','bets_all'])

bank_dog = 100.0
bet_size_dog = 1.0
bets_taken_dog = 0
df_dog = pd.DataFrame(data=[],columns=['bank_dog','bets_dog'])

bank_fav = 100.0
bet_size_fav = 1.0
bets_taken_fav = 0
df_fav = pd.DataFrame(data=[],columns=['bank_fav','bets_fav'])

bank_evplus = 100.0
bet_size_evplus = 1.0
bets_taken_evplus = 0
df_evplus = pd.DataFrame(data=[],columns=['bank_evplus','bets_evplus'])

bank_evminus = 100.0
bet_size_evminus = 1.0
bets_taken_evminus = 0
df_evminus = pd.DataFrame(data=[],columns=['bank_evminus','bets_evminus'])

bank_ev5plus = 100.0
bet_size_ev5plus = 1.0
bets_taken_ev5plus = 0
df_ev5plus = pd.DataFrame(data=[],columns=['bank_ev5plus','bets_ev5plus'])

bank_ev10plus = 100.0
bet_size_ev10plus = 1.0
bets_taken_ev10plus = 0
df_ev10plus = pd.DataFrame(data=[],columns=['bank_ev10plus','bets_ev10plus'])

bank_model1 = 100.0
bet_size_model1 = 1.0
bets_taken_model1 = 0
df_model1 = pd.DataFrame(data=[],columns=['bank_model1','bets_model1'])

bank_model2 = 100.0
bet_size_model2 = 1.0
bets_taken_model2 = 0
df_model2 = pd.DataFrame(data=[],columns=['bank_model2','bets_model2'])

for z in nfl2020.index:
    
    bet_size_all=bank_all/20
    bets_taken_all +=1
    bank_all += bet_size_all * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[nfl2020['new_date'][z], bank_all, bets_taken_all]], columns=['date','bank_all','bets_all'])
    df_all = pd.concat([df_all,temp],ignore_index=True)
    
    if nfl2020['dec'][z] > 2:
        bet_size_dog=bank_dog/20
        bets_taken_dog +=1
        bank_dog += bet_size_dog * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_dog, bets_taken_dog]], columns=['bank_dog','bets_dog'])
    df_dog = pd.concat([df_dog,temp],ignore_index=True)
    
    if nfl2020['dec'][z] < 2:
        bet_size_fav=bank_fav/20
        bets_taken_fav +=1
        bank_fav += bet_size_fav * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_fav, bets_taken_fav]], columns=['bank_fav','bets_fav'])
    df_fav = pd.concat([df_fav,temp],ignore_index=True) 
    
    if nfl2020['ev'][z] > 0:
        bet_size_evplus=bank_evplus/20
        bets_taken_evplus +=1
        bank_evplus += bet_size_evplus * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_evplus, bets_taken_evplus]], columns=['bank_evplus','bets_evplus'])
    df_evplus = pd.concat([df_evplus,temp],ignore_index=True)
    
    if nfl2020['ev'][z] < 0:
        bet_size_evminus=bank_evminus/20
        bets_taken_evminus +=1
        bank_evminus += bet_size_evminus * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_evminus, bets_taken_evminus]], columns=['bank_evminus','bets_evminus'])
    df_evminus = pd.concat([df_evminus,temp],ignore_index=True)
    
    if nfl2020['ev'][z] > 0.05:
        bet_size_ev5plus=bank_ev5plus/20
        bets_taken_ev5plus +=1
        bank_ev5plus += bet_size_ev5plus * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_ev5plus, bets_taken_ev5plus]], columns=['bank_ev5plus','bets_ev5plus'])
    df_ev5plus = pd.concat([df_ev5plus,temp],ignore_index=True)
    
    if nfl2020['ev'][z] > 0.1:
        bet_size_ev10plus=bank_ev10plus/20
        bets_taken_ev10plus +=1
        bank_ev10plus += bet_size_ev10plus * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_ev10plus, bets_taken_ev10plus]], columns=['bank_ev10plus','bets_ev10plus'])
    df_ev10plus = pd.concat([df_ev10plus,temp],ignore_index=True)
    
    if (nfl2020['ev'][z] > -0.1) & (nfl2020['ev'][z] < 0.5) & (nfl2020['qb'][z] > nfl2020['elo'][z]):
        bet_size_model1=bank_model1/20
        bets_taken_model1 +=1
        bank_model1 += bet_size_model1 * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_model1, bets_taken_model1]], columns=['bank_model1','bets_model1'])
    df_model1 = pd.concat([df_model1,temp],ignore_index=True)
    
    if (nfl2020['ev'][z] > 0.05) & (nfl2020['ev'][z] < 0.3) :
        bet_size_model2=bank_model2/20
        bets_taken_model2 +=1
        bank_model2 += bet_size_model2 * nfl2020['av'][z]
    temp = pd.DataFrame(data=[[bank_model2, bets_taken_model2]], columns=['bank_model2','bets_model2'])
    df_model2 = pd.concat([df_model2,temp],ignore_index=True)    
    
models_super_agg=pd.concat([df_all,df_dog,df_fav,df_evplus,df_evminus,df_ev5plus,df_ev10plus,df_model1,df_model2],axis=1)

In [21]:
models.to_csv('nfl2020_Bankrolls.csv')
models_agg.to_csv('nfl2020_Bankrolls_agg.csv')
models_super_agg.to_csv('nfl2020_Bankrolls_super_agg.csv')

# What Worked in 2020?

## Well, not much

First, we're gonna take a look at ROI with a fixed bet size- This will tell us the average return on any given bet that meets a set of parameters.

We're looking at what would happen if you bet the same amount every time ($1) - to obtain an average ROI per bet AKA "Actual Return per Dollar of Risk")

## This Tableau Dashboard shows which types of bets had a positive ROI on the year
### Click through the Tabs below to see ROI by different categories

In [2]:
%%html
    <div class='tableauPlaceholder' id='viz1614235267491' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;RO&#47;ROINFL2020&#47;EV&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='ROINFL2020&#47;EV' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;RO&#47;ROINFL2020&#47;EV&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1614235267491');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='1000px';vizElement.style.height='727px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

# Trying out Betting Strategies

Now that we are acquainted with which bets had a positive ROI in 2020, let's take a look at how some betting strategies would have performed in 2020

## Nine Strategies:

    1) All Bets
    
    2) All Underdog Bets
    
    3) All Favorite Bets
    
    4) All EV Positive Bets (QB ELO)
    
    5) All EV Negative Bets (QB ELO)
    
    6) All EV > 5% Bets (QB ELO)
    
    7) All EV > 10% Bets (QB ELO)
    
    8) Preferred Strategy (-10% < EV < 0.5, qbelo_prob > elo_prob)
    
    9) Backup Strategy (5% < EV < %30)

#### The Following models place all wagers with a bet size of 1% of bankroll

In [1]:
%%html
    <div class='tableauPlaceholder' id='viz1614213176446' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Co&#47;ConservativeBettingStrategyNFL2020&#47;cons&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='ConservativeBettingStrategyNFL2020&#47;cons' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Co&#47;ConservativeBettingStrategyNFL2020&#47;cons&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1614213176446');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.minWidth='800px';vizElement.style.maxWidth='1000px';vizElement.style.width='100%';vizElement.style.minHeight='587px';vizElement.style.maxHeight='727px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.minWidth='800px';vizElement.style.maxWidth='1000px';vizElement.style.width='100%';vizElement.style.minHeight='587px';vizElement.style.maxHeight='727px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='727px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

## My First Thoughts:

    1) The Control Case is "All Bets" (taking both sides of every game all season). 

    2) Betting the favorite outperformed the control- Betting the Dog did not.

    3) Strategy #4 (EV > 0) doesn't break even on the season. It outperforms the control "All Bets" strategy, but loses
    money on the year. This is important because it is the first strategy that one would be likely to try, and the strategy
    that should perform best under ideal conditions (IE you take any action that is, on paper, profitable)

    4) Strategy #6 (EV > %5) is the only profitable strategy that you might be able to reasonably guess in advance. 

    5) Strategy #8 (Preferred Strategy -10% < EV < %50, qbelo_prob > elo_prob) is our most successful model, but almost
    certainly suffers from overfitting

    6) Strategy #9 (Backup Strategy - All bets betwen 5% and 30% Expected Value) is also profitable, but does less volume
    on the season than our preferred strategy. Backup strategy is simpler. 

# What Happens when we change bet size?

#### All previous simulations were Conservative (Bet-Size = 1% of Bankroll)

Conservative: bet-size = 1% of Bankroll
    
Aggressive: bet-size = 2% of Bankroll
    
Super Aggressive: bet-size = 5% of Bankroll

## Let's Try this out on Betting Strategy #4: All bets where EV > 0

Again, this is the most important betting strategy, and the implicit optimal strategy when conditions are ideal

We want to take ANY and all (theoretically) profitable action

In [3]:
%%html
    <div class='tableauPlaceholder' id='viz1614217262836' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Be&#47;BettingStrategy4&#47;EVPlus&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='BettingStrategy4&#47;EVPlus' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Be&#47;BettingStrategy4&#47;EVPlus&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1614217262836');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

#### Strategy #4 doesn't turn a profit- but it outperforms our control (All Bets)

Strategy #4 appears to have an edge against what you'd expect if you took random bets

We're not winning yet, but the 538 EV data has positive predictive power. 

## How About Strategy #6: All Bets where EV > 5%

This is the only profitable betting strategy that relies on a single EV threshold

In [4]:
%%html
    <div class='tableauPlaceholder' id='viz1614217961050' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Be&#47;BettingStrategy6&#47;EV5Plus&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='BettingStrategy6&#47;EV5Plus' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Be&#47;BettingStrategy6&#47;EV5Plus&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1614217961050');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

#### And it's not even always profitable
Our "Super Aggressive" bet-sizing cut the bankroll so much mid-season that it finished the season under the original $100

## Strategies #8/9: Preferred and Backup Plans

#### These are actual winning strategies for the 2020 season

They are likely overfit- meaning if you follow the same strategies in 2021- I wouldn't expect them to work again. My guess is that they are due to variance

Strategy #8 (Green): -10% < EV < 50%, qbelo_prob > elo_prob

Strategy #9 (Blue): 5% < EV < 30%

In [5]:
%%html
    <div class='tableauPlaceholder' id='viz1614218667797' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Be&#47;BettingStrategy89&#47;Strategies&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='BettingStrategy89&#47;Strategies' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Be&#47;BettingStrategy89&#47;Strategies&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1614218667797');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

#### Our two preferred strategies have a pretty strong edge against the house on the NFL Season

I'm basically certain that these strategies won't hold up against next football season. The variance in the data is so high that I'm imagining there are a lot of apparent trends which can be attributed to randomness

That being said, it stands to reason that if you bet 5% of your bankroll on every bet in strategy #8, you would make a 300% return in a 6 month football season

If you were to do this, you would have made great profits, however we only know to do this with the benefit of hindsight

# Conclusions

#### 1) 538's QB ELO Probabilities appear to have some predictive power in finding more profitable sports odds
#### 2) Betting the "Out of the Box" strategy (538 Expects Value) loses money on the season
#### 3) Most of the simplest betting strategies aren't profitable, but they reduce the sportsbook's average edge
#### 4) Variance is too high to determine what trends matter- More NFL Seasons will be required to find reliable trends
#### 5) Machine learning may not be required if strong trends hold as variance reduces with bet volume